In [ ]:
import codecs
import base64
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import wordcloud

from mpl_toolkits.basemap import Basemap
from os import path
from PIL import Image

# Multiple choice data
multiple = pd.read_csv("../input/kaggle-survey-2017/multipleChoiceResponses.csv",  encoding="ISO-8859-1", low_memory=False)

# Free form data
freeform = pd.read_csv("../input/kaggle-survey-2017/freeformResponses.csv", low_memory=False)

# Goose mask base64
goose_64 = b'...'

# The Kaggle Machine Learning and Data Science Survey

---

Recently, Kaggle put together a big, fancy survey and asked people a million questions about Data Science, Machine Learning, and AI. It's an exciting survey because of the huge data science community around Kaggle. The community ranges from brand new data scientists all the way up to seasoned veterans. You can't get this level of information from any other community!

These are just a few high level items that I found interesting. Enjoy!

# What motivates people to use Kaggle?
---



In [ ]:
# Goose-shaped wordcloud

# Get the text
text = freeform[pd.notnull(freeform["KaggleMotivationFreeForm"])]["KaggleMotivationFreeForm"]

# Generate the goose mask image from base64 encoding
# Learned about base64 encoding here: https://www.kaggle.com/arthurtok/fractal-dimension-of-world-coastlines
f1 = open("goose_mask.png", "wb")
f1.write(codecs.decode(goose_64,'base64'))
f1.close()

# Goose image made into a mask
goose_file = "goose_mask.png"

# Read in the mask
goose_mask = np.array(Image.open(path.join(goose_file)))

# Generate the wordcloud
cloud = wordcloud.WordCloud(height=800, width=600, background_color="white", colormap="Blues", relative_scaling=0.2, mask=goose_mask, random_state=74364).generate(" ".join(text))

# Save to file (Only works locally)
#cloud.to_file("goosecloud.png")

# Plot wordcloud
plt.figure(figsize=(12,12))
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off");



Cool! Look how "Data science" and "Machine learning" are all standing together like a gaggle of geese!  Or A Kaggle of Data Scientists, if you will. *Just kidding, everyone knows it's pronounced "kay-gull". *

Anyway, isn't it inspiring to know that people are thirsty for knowledge around here? Of course it is! 



# Hardware
---

I'm also interested in the kind of hardware people are using. GPU's and Cloud services seem like they are all the rage these days, so maybe I need to get in on the action? In this section I pull out the answers related to the types of hardware that people use for personal projects and work projects. 

In [ ]:
# Get list of choices
personal_hardware_raw = multiple["HardwarePersonalProjectsSelect"]
work_hardware_raw = multiple["WorkHardwareSelect"]

# Split hardware choices into individual elements
def hard_split(hw):
    hardware_list = [a for answer in hw for a in re.split(r',\s*(?![^()]*\))', str(answer))]
    return hardware_list

personal_hardware_split = hard_split(personal_hardware_raw)
work_hardware_split = hard_split(work_hardware_raw)

# Dict to make questions less specific
hardware_map = {
    "Basic laptop (Macbook)": "Basic Laptop",
    "Gaming Laptop (Laptop + CUDA capable GPU)": "Laptop with GPU",
    "Laptop or Workstation and local IT supported servers": "Laptop/Desktop with local servers",
    "Laptop or Workstation and private datacenters": "Laptop/Desktop with private datacenter",
    "Traditional Workstation": "Desktop",
    "GPU accelerated Workstation": "Desktop with GPU",
    "Laptop + Cloud service (AWS, Azure, GCE ...)":  "Laptop + Cloud Service",
    "Workstation + Cloud service": "Desktop + Cloud Service",
    "Other": "Other",
    "nan": np.nan
}

# Change the questions now that everyone has answered them 
def hard_general(hw):
    hardware = [hardware_map[h] for h in hw]
    return hardware

personal_hardware = hard_general(personal_hardware_split)
work_hardware = hard_general(work_hardware_split)

# Hardware subplots
ph = pd.DataFrame(data=personal_hardware, columns=["personal"])
personal = ph["personal"].value_counts()

wh = pd.DataFrame(data=work_hardware, columns=["work"])
work = wh["work"].value_counts()

y_pos = np.arange(len(work))
y_pos2 = np.arange(len(personal))

fig,(ax1,ax2) = plt.subplots(2, sharex=False, figsize=(8,10))

# Personal hardware
ax1.barh(bottom=y_pos2, width=personal, color='royalblue')
ax1.set_yticks(y_pos)
ax1.set_yticklabels(personal.index)
ax1.invert_yaxis()  # labels read top-to-bottom
ax1.set_title('Hardware used in personal projects')
fig.subplots_adjust(hspace=0.25)

# Work hardware
ax2.barh(bottom=y_pos, width=work, color='royalblue')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(work.index)
ax2.invert_yaxis()  # labels read top-to-bottom
ax2.set_title('Hardware used in work projects')

ax2.set_xlabel("Number of responses", fontsize=10)

plt.show()

Laptops appear to be the most used piece of hardware in data science. It's true for both work-related and personal projects. You don't usually think of laptops as huge computational powerhouses so why are so many people using them? At work you probably have more resources and could have a desktop with 50 GPUs but it probably makes more sense to use cloud services or local servers. At home you don't have those same resources so it's more likely a trade off between watching Netflix in bed or training models - I know which one I would choose (Watching Netflix in bed with a Desktop full of GPUs, obviously).



# Languages, Software, and Tools
---

The argument of which programming language is best is probably older than time. We probably can't answer that question but we can at least see what languages people use  the most!

In [ ]:
# What algorithms and methods do data scientists use at work
work = pd.DataFrame()
work["algorithms"] = multiple.WorkAlgorithmsSelect
work["tools"] = multiple.WorkToolsSelect
work["methods"] = multiple.WorkMethodsSelect
work["nextyear"] = multiple.MLToolNextYearSelect

# Prep tool data
work.dropna(inplace=True)
# Separate choices
work_tools = hard_split(work["tools"])
work_tools = pd.DataFrame(data=work_tools, columns=["tools"])

# Plot languages, tools, services
wt = work_tools["tools"].value_counts()
ax = wt.plot(kind="barh", figsize=(12,12), color="royalblue")
ax.invert_yaxis()
ax.set_title("Tools - Languages, Software, and Services");

Python, R, and SQL come out as the three most used languages in the survey. This is really valuable information for any new data scientists out there. I'd say start learning what's at the top and work your way down :) 

# What people want to learn
---

As we already know, lots of people are coming to Kaggle to learn new things. One of the survey questions asked people to choose one thing that they are excited to learn about in the next year. 

In [ ]:
# What do people want to learn next year
future = pd.DataFrame(data=multiple.MLToolNextYearSelect)
counts = future.MLToolNextYearSelect.value_counts()
ax = counts.plot(kind="barh", figsize=(12,12), color="royalblue")
ax.invert_yaxis()
ax.set_title("What people want to learn");

Tensorflow is really high up on the list! However, Tensorflow was the only deep learning framework option availble in the survey. I would be willing to wager that people are probably also excited about other frameworks such as Caffe, CNTK, MXNet, and Torch. But suffice it to say, people are definitely excited about deep learning tools. Python and R are also high on the list of things people want to learn and there are a lot of exciting cloud service offerings that people want to check out.. 

Most importantly though, high-five to the 13 of you excited to learn Perl this year :D 

# How important is your Kaggle ranking for getting a job?
---


In [ ]:
kr = pd .DataFrame(data=multiple.JobSkillImportanceKaggleRanking)
kr.dropna(inplace = True)
counts = kr.JobSkillImportanceKaggleRanking.value_counts()
for i in range(0,len(counts)):
    print(counts.index[i], "\t", str(round(counts[i] / len(kr) * 100, 1)) +"%")

# Top 10 industries that data scientists work in
---

In [ ]:
df = pd.DataFrame(data=multiple.EmployerIndustry)
counts = df.EmployerIndustry.value_counts()[:10]
for i in range(0,len(counts)):
    print(str(round(counts[i] / len(kr) * 100, 1))+"%", "\t", counts.index[i])

# Which is more important to know, Python or R? 
---

In [ ]:
sp = pd.DataFrame(data=multiple.JobSkillImportancePython)
sr = pd.DataFrame(data=multiple.JobSkillImportanceR)
sp.head()
sr.head()
job_skill = pd.merge(sp, sr, left_index=True, right_index=True, how="outer")
job_skill.dropna(axis=0, thresh=2, inplace=True)
python_importance_counts = round(job_skill["JobSkillImportancePython"].value_counts()/len(job_skill["JobSkillImportancePython"]) *100, 1)
print("Python skill importance:")
for i in range(0,len(python_importance_counts)):
    print(python_importance_counts.index[i], "\t", str(python_importance_counts[i])+"%")
    
r_importance_counts = round(job_skill["JobSkillImportanceR"].value_counts()/len(job_skill["JobSkillImportanceR"]) * 100, 1)
print("\nR skill importance")
for i in range(0, len(r_importance_counts)):
    print(r_importance_counts.index[i], "\t", str(r_importance_counts[i])+"%")

# What countries are people from?
---

In [ ]:
# Make the country names same as lat/lon dataset
def swap_country(country):
    country = str(country)    
    if "United States" in country:
        return "US"
    elif "United Kingdom" in country:
        return "UK"
    elif "China" in country:
        return "China" # Multiple versions of People's republic of China, Republic of China, etc.)
    elif "nan" in country:
        return np.nan
    else: 
        return country
    
# Relative point size for the map
def point_size(count):
    point_size = count + 300  
    return point_size

# Read in lat/lon dataset
coords = pd.read_csv("../input/world-countries-and-continents-details/Countries Longitude and Latitude.csv", index_col=0, header=0, skiprows=2, names=["latitude", "longitude", "name"])

# Change country names to match lat/lon dataset
country_corrected = multiple.Country.apply(lambda x: swap_country(x))

# Get the number of responses from each country
country_counts = country_corrected.value_counts()

# lat/lon dicts
latmap = dict(zip(coords['name'], coords['latitude']))
lonmap = dict(zip(coords['name'], coords['longitude']))

# make the df
df = pd.DataFrame()
df["name"] = country_counts.index
df["counts"] = country_counts.values
df["latitude"] = df["name"].map(latmap)
df["longitude"] = df["name"].map(lonmap)
df["point_size"] = df["counts"].apply(lambda x: point_size(x))

# Turn off warnings
pd.options.mode.chained_assignment = None

# Change US lat/lon to geographic center
# The orignal points to New York and I've never even been to New York
df.latitude[df.name == "US"] = 39.8283
df.longitude[df.name == "US"] = -98.5795

plt.figure(figsize=(24,24))

# Build the basic map
my_map = Basemap(projection='robin', lat_0=70, lon_0=20, resolution='h')
my_map.drawcoastlines()
my_map.drawcountries()
my_map.bluemarble()
my_map.drawmapboundary(fill_color="white")

# Prep the coordinates for projection
for country in df.name:
    lon = df.longitude[df.name == country].values
    lat = df.latitude[df.name == country].values
    x,y = my_map(lon,lat)
    point_size = df.point_size[df.name == country]
    my_map.scatter(x, y, latlon=False, marker="o", s=point_size, color="#20BEFF", zorder=10, edgecolor="#000000", linewidth=1)
    
plt.title("Distribution of Survey Respondents", fontsize=30)
plt.show()

# Conclusion
---

We saw in the Goose-shaped word cloud that people are motivated to use Kaggle to learn new things. We also saw that most people use laptops as their main tool, Python and R for their programming languages, and data scientists come from all over the world! If you're new to data science, take a look at these lists and pick a few of the top results to go learn! 

There's my brief look at the 2017 Data Science and Machine Learning Survey results. I had a great time analyzing it and I hope you found it useful. Cheers!